Importing neccessary package

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

y_prime(y) is result of function dy = f(x, y(x)), 'bar5.csv' is the sample data of the solution upto x = 3

In [ ]:
def y_prime(y):
  return 0.5 * y + 0.25

In [ ]:
ode_dataset = pd.read_csv('bar5.csv')

# plt.plot(ode_dataset.t, ode_dataset.x)
# plt.plot(ode_dataset.t, ode_dataset.y)

train_dataset = ode_dataset.sample(frac=0.2, random_state=0)
test_dataset = ode_dataset.drop(train_dataset.index)


In [ ]:
train_labels = train_dataset.copy()
test_labels = test_dataset.copy()

train_features = train_labels.pop('t')
test_features = test_labels.pop('t')


In [ ]:
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())

In [ ]:
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(20, activation='relu'),               # Hidden 1
    layers.Dense(20, activation='relu'),               # Hidden 2
    layers.Dense(20, activation='relu'),               # Hidden 3
    layers.Dense(20, activation='relu'),               # Hidden 4
    layers.Dense(20, activation='relu'),               # Hidden 5
    # layers.Dense(20, activation='relu'),               # Hidden 6
    # layers.Dense(20, activation='relu'),               # Hidden 7
    # layers.Dense(20, activation='relu'),               # Hidden 8
    # layers.Dense(20, activation='relu'),               # Hidden 9
    # layers.Dense(1)                                    # For single output
    layers.Dense(1)
])

model.summary()

In [ ]:
loss_ODE = tf.keras.losses.MeanSquaredError()
loss_boundary = tf.keras.losses.MeanSquaredError()

def loss(model, x, y, training):
  x = tf.Variable(x)
  with tf.GradientTape() as t:
    t.watch(x)
    y_ = model(x, training=training)    # Prediction

  # ODE
  dy__dx = t.gradient(y_, x)
  y_prime_ = tf.Variable(y_prime(y_))    # Single output
  # y_prime_ = tf.Variable(y_prime(y_[0], y[1]))

  return tf.reduce_mean(tf.dtypes.cast(loss_ODE(y_prime_, dy__dx), tf.float32)) + \
         tf.reduce_mean(tf.dtypes.cast(loss_boundary( y_, y ), tf.float32))
  #  

l = loss(model, train_features.max(), train_labels.max(), training=False)
print("Loss train: {}".format(l))


In [ ]:
# Update weights
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

# Choose optimizer
optimizer = tf.keras.optimizers.Adam()

Training loop

In [ ]:
train_input = np.vstack([train_features, train_labels.T]).T

In [ ]:
# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

In [ ]:
epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.MeanSquaredError()

num_epochs = 21

for epoch in range(num_epochs):
  # Metrics

  # Training loop - using batches of 32
  for apair in train_input:
    x = apair[0]
    y = apair[1:]

    # Optimize the model
    loss_value, grads = grad(model, x, y)
    grads = [grad if grad is not None else tf.zeros_like(var)
    for var, grad in zip(model.trainable_variables, grads)]
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss

    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy.update_state(y, model(x, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 5 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Plotting

In [ ]:
# fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
# fig.suptitle('Training Metrics')

# axes[0].set_ylabel("Loss", fontsize=14)
plt.plot(train_loss_results)
plt.title('Training Metrics')
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)

# axes[1].set_ylabel("Accuracy", fontsize=14)
# axes[1].set_xlabel("Epoch", fontsize=14)
# axes[1].plot(train_accuracy_results)
plt.show()

In [ ]:
# If you decide to load old model
# new_model = tf.keras.models.load_model('/first_function_PINN.h5')
# new_model.summary()


In [ ]:

ode_plot_dataset = pd.read_csv('bar5.csv')
whole_dataset = np.vstack([ode_plot_dataset.t, ode_plot_dataset.x]).T
# whole_dataset = np.vstack([ode_plot_dataset.t, ode_plot_dataset.x, ode_plot_dataset.y]).T

for_plot1 = []
# for_plot2 = []

for pairs in whole_dataset:
  result1 = model(pairs[0], training=False)[0]
  for_plot1.append(result1)

  # For 2 outputs

  # result1 = model(pairs[0], training=False)[0][0]
  # for_plot1.append(result1)
  # result2 = model(pairs[0], training=False)[0][1]
  # for_plot2.append(result2)


In [ ]:
plt.figure(figsize=(16, 8))

plt.plot(ode_plot_dataset.t, whole_dataset[:, 1], 'b',  linewidth=4, label='exact')
plt.plot(ode_plot_dataset.t, for_plot1, 'r:', linewidth=4,
                                                dash_capstyle='round', label='predict')
# plt.plot(ode_plot_dataset.t, whole_dataset[:, 2], 'b',  linewidth=4, label='exact_y')
# plt.plot(ode_plot_dataset.t, for_plot2, 'r:', linewidth=4,
#                                                 dash_capstyle='round', label='predict_y')
plt.title('dy/dx = 0.5y + 0.25')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
# model.save('name.h5')
